In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [8]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),                     # Horizontally flip the given image randomly with a given probability
                                transforms.RandomResizedCrop((32, 32)),                # Crop a random portion of image and resize it to a given size
                                transforms.RandomRotation(15),                         # Rotate the image by angle
                                transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), # Random affine transformation of the image keeping center invariant
                                transforms.RandomInvert(p=0.3),                        # Inverts the colors of the given image randomly with a given probability
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize
                               ])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                         download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                           shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("using", device)

using cuda


In [10]:
class Network(nn.Module):
  def __init__(self, n_in=3,
               n_hidden1=4,
               n_hidden2=8,
               n_hidden3=16,
               n_hidden4=32,
               n_hidden5=64,
               n_hidden6=128,
               n_l1=1000,
               n_l2=100,
               n_output=10,
               kernel_size_cnn=3,
               kernel_size_pool=2,
               p=0.2,
               padding=1):
    super().__init__()
    self.conv_layer = nn.Sequential(
        nn.Conv2d(n_in, n_hidden1, kernel_size_cnn, padding=padding), #(in_channels, out_channels, kernel_size)
        nn.BatchNorm2d(n_hidden1), #(number of features)
        nn.ReLU(),
        nn.Conv2d(n_hidden1, n_hidden2, kernel_size_cnn, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size_pool),

        nn.Conv2d(n_hidden2, n_hidden3, kernel_size_cnn, padding=padding),
        nn.BatchNorm2d(n_hidden3),
        nn.ReLU(),
        nn.Conv2d(n_hidden3, n_hidden4, kernel_size_cnn, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size_pool),
        nn.Dropout(p),

        nn.Conv2d(n_hidden4, n_hidden5, kernel_size_cnn, padding=padding),
        nn.BatchNorm2d(n_hidden5),
        nn.ReLU(),
        nn.Conv2d(n_hidden5, n_hidden6, kernel_size_cnn, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size_pool), # output shape = (128, 4, 4)
    )

    self.fc_layer = nn.Sequential(
        nn.Dropout(p),
        nn.Flatten(),
        nn.Linear(n_hidden6*4*4, n_l1),
        nn.ReLU(),
        nn.Linear(n_l1, n_l2),
        nn.ReLU(),
        nn.Dropout(p),
        nn.Linear(n_l2, n_output)
    )

  def forward(self, x):
    x = self.conv_layer(x)
    x = self.fc_layer(x)

    return x

model = Network().to(device)
print(model)

Network(
  (conv_layer): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout(p=0.2, inplace=False)
    (13): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU()
    (16): Conv2d(64, 128, kernel_s

In [16]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.005)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  correct = 0
  model.train()
  for batch, (x, y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()

    pred = model(x)
    loss = loss_fn(pred, y)
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    if batch % 500 == 0:
        loss, current = loss.item(), batch * batch_size
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

  print(f"total train accuracy: {correct*100/size:>7f}")

def test(dataloader, model, loss_fn, optimizer):
  correct_pred = {c: 0 for c in classes}
  total = {c: 0 for c in classes}

  model.eval()
  with torch.no_grad():
    for data in dataloader:
      x, y = data
      x, y = x.to(device), y.to(device)
      out = model(x)
      pred = out.argmax(1)
      for y, pred in zip(y, pred):
        if y == pred:
          correct_pred[classes[y]] += 1
        total[classes[y]] += 1
  
  # print results
  for c in classes:
    acc = float(correct_pred[c]) / total[c]
    print(f"Accuracy of {c}: {acc*100}%")

for epoch in range(2):
  print(f"Epoch {epoch+1}\n-------------------------------")
  train(train_loader, model, loss_fn, optimizer)
print('Finished Training')

test(test_loader, model, loss_fn, optimizer)


Epoch 1
-------------------------------
loss: 2.275115  [    0/50000]
loss: 1.925573  [ 2000/50000]
loss: 1.852545  [ 4000/50000]
loss: 2.354488  [ 6000/50000]
loss: 1.852010  [ 8000/50000]
loss: 1.970184  [10000/50000]
loss: 1.995650  [12000/50000]
loss: 2.248348  [14000/50000]
loss: 2.021744  [16000/50000]
loss: 2.040428  [18000/50000]
loss: 2.993144  [20000/50000]
loss: 2.113527  [22000/50000]
loss: 2.301047  [24000/50000]
loss: 1.972690  [26000/50000]
loss: 2.044314  [28000/50000]
loss: 2.266176  [30000/50000]
loss: 2.218407  [32000/50000]
loss: 2.179538  [34000/50000]
loss: 1.844463  [36000/50000]
loss: 1.995457  [38000/50000]
loss: 1.576590  [40000/50000]
loss: 2.082208  [42000/50000]
loss: 1.965498  [44000/50000]
loss: 2.110747  [46000/50000]
loss: 2.166228  [48000/50000]
total train accuracy: 22.540000
Epoch 2
-------------------------------
loss: 2.030478  [    0/50000]
loss: 2.009756  [ 2000/50000]
loss: 1.941110  [ 4000/50000]
loss: 1.688848  [ 6000/50000]
loss: 2.260008  [ 